In [ ]:
# Перед работой необходимо вручную скачать файлы:
# total_reports.csv
# posts_spb_today.csv
# kg.graphml
# Ссылка для скачивания файлов: https://drive.google.com/drive/folders/1iOtyu8P1eUTBTVvfyDK4dpTxQ4cuPmxR?usp=drive_link

In [1]:
!pip install nx

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=6b6326b60dab1a817c0aae9422e45702aaef0c10050e9b05edbaf22a9f64c853
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
import csv
import zipfile
import re
import gensim
import logging
import nltk.data
import pandas as pd
import numpy as np
import networkx as nx
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from string import ascii_letters as al
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Скачиваем модель
urllib.request.urlretrieve("http://vectors.nlpl.eu/repository/20/220.zip", '220.zip')

('220.zip', <http.client.HTTPMessage at 0x7fb484c0ea10>)

In [4]:
# Распаковываем модель
with zipfile.ZipFile('220.zip', 'r') as zip_ref:
    zip_ref.extract('model.bin', 'models')

In [5]:
# Открываем модель
model_path = 'models/model.bin'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [6]:
# Тестируем модель
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'биткоин_NOUN']

In [7]:
for word in words:
    # есть ли слово в модели?
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_NOUN
[ 1.607006    0.88105875 -0.90222335  0.1547316   2.5412288  -0.7520914
  1.7317989   0.5731876  -2.1542902  -2.353275  ]
неделя_NOUN 0.767143726348877
месяц_NOUN 0.7355299592018127
утро_NOUN 0.6559638381004333
час_NOUN 0.6324191689491272
ночь_NOUN 0.5809687972068787
сутки_NOUN 0.5776057243347168
вечер_NOUN 0.5675091743469238
минута_NOUN 0.548324465751648
день»_PROPN 0.5270609259605408
денька_NOUN 0.515771210193634


ночь_NOUN
[-1.9102603  -1.5677836   0.5107194   0.7136092  -1.7047127  -4.622845
  3.485842    1.2509006   0.04754175 -1.8432467 ]
вечер_NOUN 0.7615349292755127
утро_NOUN 0.7531794309616089
рассвет_NOUN 0.727114737033844
полночь_NOUN 0.6713059544563293
полдень_NOUN 0.6476588249206543
сумерки_NOUN 0.60179603099823
утр_NOUN 0.5851607322692871
день_NOUN 0.5809688568115234
темнота_NOUN 0.5586055517196655
ночной_ADJ 0.5421528220176697


человек_NOUN
[-3.8098485   2.7259772   1.1949589  -1.3183012   2.5352347  -1.1812418
  0.43386376 -0.49407318 -2.751272   -3.2406285 

In [8]:
print(model_ru.similarity('снег_NOUN', 'тротуар_NOUN'))

0.400761


In [ ]:
# Обучаем свою модель

In [9]:
# Переписываем данные в текстовый файл
with open('comments.txt', 'a', encoding='utf-8') as file:
  with open('posts_spb_today.csv', 'r', encoding='utf-8') as data:
    for row in data:
      file.write(row)

In [10]:
# Подчищаем файл
with open('comments.txt') as f:
    data = ''.join(i for i in f.read() if not i.isdigit())

with open('comments.txt', 'w') as f:
    f.write(data)

In [11]:
# Преобразуем текст в список из списков слов для каждого предложения [[Слово_1, Слово_2,...],[Слово_1, Слово_2,...],...]
with open("comments.txt", 'r', encoding='utf-8') as f:
    text = f.read()

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)

print(clean_sents[:2])

[['date;text;id;link', '', 'в', 'санкт-петербурге', 'продолжается', 'расследование', 'уголовного', 'дела', 'возбужденного', 'по', 'факту', 'массового', 'убийства', 'мирных', 'граждан', 'в', 'годы', 'великой', 'отечественной', 'войныв', 'рамках', 'работы', 'проводимой', 'сотрудниками', 'гсу', 'ск', 'россии', 'по', 'г', 'санкт-петербургу', 'по', 'установлению', 'преступлений', 'совершенных', 'немецко-фашистскими', 'захватчиками', 'в', 'период', 'оккупации', 'ленинграда', 'в', '', 'гг', 'во', 'взаимодействии', 'с', 'сотрудниками', 'центрального', 'государственного', 'архива', 'санкт-петербурга', 'были', 'получены', 'копии', 'архивных', 'документов', 'изданные', 'в', 'городе', 'павловске', 'пушкинского', 'района', 'во', 'время', 'великой', 'отечественной', 'войны', 'немецкими', 'оккупационными', 'властями'], ['в', 'указанных', 'документах', 'фиксировались', 'факты', 'расстрелов', 'гражданского', 'населения', 'машинописным', 'текстом', 'с', 'рукописными', 'подписями', 'исполнителей']]


In [12]:
# Создаём модель
from gensim.models import Word2Vec

model = Word2Vec(clean_sents, vector_size = 10, window=5,min_count = 1)

In [13]:
# Загружаем данные для обучения
data = pd.read_csv("posts_spb_today.csv", delimiter=";")

len(data)

8392

In [14]:
data.head()

,Unnamed: 0,date,text,id,link
0,0,2023-11-21 14:47:17,В Санкт-Петербурге продолжается расследование ...,18541310,NaN
1,1,2023-11-21 14:05:01,Водитель Порше обиделсяПредыстория:После перес...,18541140,NaN
2,2,2023-11-21 14:03:04,Мерседес сбил доставщика на электропеде на пер...,18541135,NaN
3,3,2023-11-21 14:00:04,"На КАД'е в аэродрома Левашово, произошло жёстк...",18541124,NaN
4,4,2023-11-21 13:00:01,"— Данный пост будет интересен тем, у кого в кв...",18541019,NaN


In [15]:
# Функции для очистки датасета

tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[f-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^а-яА-Я]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = stopwords.words("russian")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [16]:
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []

print("Parsing sentences from training set...")
for review in data["text"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


<ipython-input-15-075fad4dee11>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review, "lxml").get_text()
<ipython-input-15-075fad4dee11>:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review, "lxml").get_text()


In [17]:
print(len(sentences))
print(sentences[0])

26002
['в', 'санкт', 'петербурге', 'продолжается', 'расследование', 'уголовного', 'дела', 'возбужденного', 'по', 'факту', 'массового', 'убийства', 'мирных', 'граждан', 'в', 'годы', 'великой', 'отечественной', 'войныв', 'рамках', 'работы', 'проводимой', 'сотрудниками', 'гсу', 'ск', 'россии', 'по', 'г', 'санкт', 'петербургу', 'по', 'установлению', 'преступлений', 'совершенных', 'немецко', 'фашистскими', 'захватчиками', 'в', 'период', 'оккупации', 'ленинграда', 'в', 'гг', 'во', 'взаимодействии', 'с', 'сотрудниками', 'центрального', 'государственного', 'архива', 'санкт', 'петербурга', 'были', 'получены', 'копии', 'архивных', 'документов', 'изданные', 'в', 'городе', 'павловске', 'пушкинского', 'района', 'во', 'время', 'великой', 'отечественной', 'войны', 'немецкими', 'оккупационными', 'властями']


In [18]:
# Тренировка модели
print("Training model...")

%time model_comments = word2vec.Word2Vec(sentences, workers=4, vector_size=300, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 4.9 s, sys: 36.5 ms, total: 4.94 s
Wall time: 3.82 s


In [19]:
print(len(model_comments.wv.key_to_index))

4064


In [ ]:
# Проверка модели
model_comments.wv.similarity('снег', 'тротуар')

0.9607089

In [20]:
# Загрузка даных для дообучения
train_data = pd.read_csv("total_reports.csv", delimiter=";")

train_data.dropna(subset='Текст', inplace=True)
len(train_data)
train_data['Текст'].head(1045)

0       Тоже поздравляю всех мужчин нашего района 😊 От...
1       У нас вообще заклеили скотчем спи...дил Батаре...
2       Добрый день, вроде бы праздник сегодня, но поч...
3       Доброе утро! Сегодня 23 февраля. Жкс2 снова вс...
4       [id4346392:bp-145945152_963|Тамара], нужно у С...
                              ...                        
1040    Добрый день! Когда у нас будут ЖК выполнять св...
1042    Добрый вечер Уважаемый Александр Дмитриевич , ...
1043            А снег когда уберете? 3 месяца куча лежит
1044    [id144340|Ольга], ну некоторые просто или прав...
1045    Вычистите дом по адресу ул.Белы Куна дом 20 к....
Name: Текст, Length: 1045, dtype: object

In [21]:
# Токенизация данных
sentences = []

print("Parsing sentences from training set...")
for review in train_data["Текст"]:
  sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


<ipython-input-15-075fad4dee11>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review, "lxml").get_text()
<ipython-input-15-075fad4dee11>:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review, "lxml").get_text()


In [22]:
# Сохранения модели перед дообучением
model_path = "comments.model"

print("Saving model...")
model_comments.save(model_path)

Saving model...


In [23]:
# Тренировка модели
model_trained = gensim.models.Word2Vec.load(model_path)

model_trained.build_vocab(clean_sents, update=True)
model_trained.train(sentences, total_examples=model_trained.corpus_count, epochs=5)

(2795787, 5321580)

In [24]:
# Проверка модели
model_trained.wv.similarity('снег', 'тротуар')

0.83836246

In [ ]:
# Пробуем использовать сразу 2 модели

In [25]:
static_result = model_ru.similarity('снег_NOUN', 'тротуар_NOUN')

custom_result = model_trained.wv.similarity('снег', 'тротуар')

semantic_closeness = (static_result + custom_result)/2
print(semantic_closeness)


0.6195617318153381


In [ ]:
# Достаём пары из графов и оцениваем семантическую близость слов

In [28]:
G = nx.read_graphml('kg.graphml')

In [29]:
df = nx.to_pandas_edgelist(G)

In [89]:
df.head(100)

,source,target,verb
0,Мерседес,доставщика электропеде,сбил
1,Мерседес,сторону реки,покатился
2,Мерседес,Буграх,загорелся
3,Мерседес,бус,задела
4,Мерседес,проспекте,перевернулся
...,...,...,...
95,Задержанны,ответственности хулиганство,привлекался
96,Мошенники,людей,убеждали
97,Фальшивомонетчики,их,продавали
98,Рахимов,стоянку,осуществил


In [121]:
# Функция отбрасывания английских слов
def expectlatin(mystring):
  result = ''
  for i in mystring:
    if i not in al:
      result += i
  if result != '':
    return result
  else:
    return 'абв'

# Запись оценки в csv
with open('semantic_value.csv', mode='w', encoding='utf-8') as file:
  sucess = 0
  fail = 0
  names = ['source', 'target', 'semantic closeness']

  file_writer = csv.DictWriter(file, delimiter = ";", lineterminator="\r", fieldnames=names)

  for i in range(len(df)):

    # список из пары слов
    words = [df['source'][i].lower(), df['target'][i].lower()]
    # оставляем только буквы
    words = [expectlatin(''.join(c for c in word if c.isalpha())) for word in words]
    # проверка на наличие слова в модели
    for word in words:
      # есть ли слово в модели?
      if word in model_trained.wv.key_to_index:
        check_status = True
      else:
        fail += 1
        check_status = False
        # print('Увы, слова "%s" нет в модели!' % word)
        break
    # оценка пар слов
    if check_status == True:
      sucess += 1
      print(words)
      # static_result = model_ru.similarity(f"{word[0]}_NOUN", f"{word[1]}_NOUN")
      custom_result = model_trained.wv.similarity(word[0], word[1])
      # semantic_closeness = (static_result + custom_result)/2
      print(custom_result)
      file_writer.writerow({'source': words[0], 'target': words[1], 'semantic closeness': custom_result})


  print(f'Успешных оценок: {sucess}')
  print(f'Провальных оценок: {fail}')

['мерседес', 'буграх']
0.029653393
['мерседес', 'проспекте']
0.671775
['мерседес', 'ребенка']
0.48111013
['легковушка', 'автобус']
0.10541714
['петербуржец', 'голову']
0.44207928
['петербуржец', 'месяцев']
0.51807183
['петербуржец', 'больнице']
0.26772082
['матиз', 'направлении']
0.11520758
['фура', 'жд']
0.35025305
['скорая', 'помощь']
0.4898747
['ло', 'дома']
0.16731389
['опель', 'аллее']
0.24383055
['солярис', 'карш']
0.13387494
['солярис', 'полосу']
0.4898747
['солярис', 'скорой']
0.13987514
['солярис', 'бок']
0.26772082
['солярис', 'мотоциклиста']
0.14983714
['солярис', 'транспорт']
0.25404608
['солярис', 'сампсониевском']
0.05072916
['солярис', 'мотоцикл']
0.14983714
['полиция', 'петербуржцу']
0.32585424
['полиция', 'проверку']
0.671775
['полиция', 'проверку']
0.671775
['полиция', 'проверку']
0.671775
['полиция', 'проверку']
0.671775
['полиция', 'проверку']
0.671775
['полиция', 'проверку']
0.671775
['полиция', 'таксиста']
0.2978334
['полиция', 'подростков']
0.4898747
['полиция', 